In [1]:
import numpy as np, pandas as pd
from datetime import datetime
from langdetect import detect

At this moment we have 3 .csv files for EACH platform:
<br/>1- Games information
<br/>2- Critics reviews
<br/>3- Users reviews
<br/>Total: 9 files
<br/><br/>The objective is to have just 2 cleaned .csv files: 
<br/>1- Games information of all platforms
<br/>2- Reviews (critics + users) of all platforms

## Games info

#### PS4

We first need to load the .csv file

In [2]:
df = pd.read_csv('data/ps4_games.csv', lineterminator='\n')

Let's take a look at first 5 rows

In [5]:
df.head().transpose()

,0,1,2,3,4
title,10 Second Ninja X,1000 Top Rated,1001 Spikes,100ft Robot Golf,101 Ways to Die
platform,PlayStation 4,PlayStation 4,PlayStation 4,PlayStation 4,PlayStation 4
developer,GameDesignDan,Top Rated,Nicalis,No Goblin,4 Door Lemon
genre,"['Action', 'Platformer', '2D']","['Miscellaneous', 'Party / Minigame']","['Action', 'Platformer', 'Platformer', '2D', '...","['Sports', 'Individual', 'Golf', 'Arcade']","['Action', 'Platformer', '2D']"
rating,T,NaN,T,E10+,T
release_date,"Jul 19, 2016","Jun 27, 2017","Jun 3, 2014","Sep 27, 2017","Mar 22, 2016"
summary,If you want a cheesy story to go along with a ...,"This game is a puzzle game, but it's more like...",If you miss the crippling difficulty of the NE...,100ft Robot Golf is a real-time golf game thro...,A game that makes me laugh and want to play mo...
meta_score,76,0,78,56,60
meta_overview,Generally favorable reviews,No score yet,Generally favorable reviews,Mixed or average reviews,Mixed or average reviews
meta_pos,11,0,6,3,1


Are there missing values ? what about formats?

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2704 entries, 0 to 2703
Data columns (total 17 columns):
title            2704 non-null object
platform         2704 non-null object
developer        2695 non-null object
genre            2704 non-null object
rating           2428 non-null object
release_date     2704 non-null object
summary          2396 non-null object
meta_score       2704 non-null int64
meta_overview    2703 non-null object
meta_pos         2703 non-null float64
meta_mixed       2703 non-null float64
meta_neg         2703 non-null float64
user_score       2703 non-null object
user_overview    2703 non-null object
user_pos         2703 non-null object
user_mixed       2703 non-null object
user_neg         2703 non-null object
dtypes: float64(3), int64(1), object(13)
memory usage: 380.2+ KB


The table has 2704 entries. There are rows with missing values.

The columns user_score, user_pos, user_mixed and user_neg must have numbers but their format is "object". They need to be transformed. Null values are replaced with 0.

In [7]:
def obj_to_numeric(df, cols):
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df.loc[df[col].isnull(), col] = 0

In [8]:
obj_to_numeric(df, ['user_score', 'user_pos', 'user_mixed', 'user_neg'])

Fill meta_overview and user_overview null values with their corresponding categories when there's no score.

In [9]:
df.loc[df['meta_overview'].isnull(), 'meta_overview'] = 'No score yet'
df.loc[df['user_overview'].isnull(), 'user_overview'] = 'No user score yet'

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2704 entries, 0 to 2703
Data columns (total 17 columns):
title            2704 non-null object
platform         2704 non-null object
developer        2695 non-null object
genre            2704 non-null object
rating           2428 non-null object
release_date     2704 non-null object
summary          2396 non-null object
meta_score       2704 non-null int64
meta_overview    2704 non-null object
meta_pos         2704 non-null float64
meta_mixed       2704 non-null float64
meta_neg         2704 non-null float64
user_score       2704 non-null float64
user_overview    2704 non-null object
user_pos         2704 non-null float64
user_mixed       2704 non-null float64
user_neg         2704 non-null float64
dtypes: float64(7), int64(1), object(9)
memory usage: 380.2+ KB


Let's see meta_score and user_score min and max values. We are looking for outliers.

In [11]:
df.describe().loc[['min', 'max'], ['meta_score', 'user_score']]

,meta_score,user_score
min,0.0,0.0
max,97.0,9.6


No outliers but the columns are in different range. They should be in the same range in order to be able to compare them.

In [12]:
df['n_user_score'] = df['user_score'] * 10

release_date column contain strings, for instance "Jul 19, 2016". We must transform those strings into datetime.

In [13]:
df['release_date'] = df['release_date'].apply(lambda x: pd.to_datetime(datetime.strptime(x, '%b %d, %Y')))

Save cleaned dataframe

In [14]:
df.to_csv('ps4_games_cleaned.csv', index=False, encoding = 'utf-8')

#### Xbox One

Apply the same steps for Xbox One and Switch game information files

In [15]:
df = pd.read_csv('xboxone_games.csv', lineterminator='\n')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1976 entries, 0 to 1975
Data columns (total 17 columns):
title            1976 non-null object
platform         1976 non-null object
developer        1967 non-null object
genre            1976 non-null object
rating           1636 non-null object
release_date     1976 non-null object
summary          1596 non-null object
meta_score       1976 non-null int64
meta_overview    1975 non-null object
meta_pos         1975 non-null float64
meta_mixed       1975 non-null float64
meta_neg         1975 non-null float64
user_score       1975 non-null object
user_overview    1975 non-null object
user_pos         1975 non-null object
user_mixed       1975 non-null float64
user_neg         1975 non-null float64
dtypes: float64(5), int64(1), object(11)
memory usage: 277.9+ KB


In [18]:
obj_to_numeric(df, ['user_score', 'user_pos', 'user_mixed', 'user_neg'])

In [19]:
df.loc[df['meta_overview'].isnull(), 'meta_overview'] = 'No score yet'
df.loc[df['user_overview'].isnull(), 'user_overview'] = 'No user score yet'

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1976 entries, 0 to 1975
Data columns (total 17 columns):
title            1976 non-null object
platform         1976 non-null object
developer        1967 non-null object
genre            1976 non-null object
rating           1636 non-null object
release_date     1976 non-null object
summary          1596 non-null object
meta_score       1976 non-null int64
meta_overview    1976 non-null object
meta_pos         1976 non-null float64
meta_mixed       1976 non-null float64
meta_neg         1976 non-null float64
user_score       1976 non-null float64
user_overview    1976 non-null object
user_pos         1976 non-null float64
user_mixed       1976 non-null float64
user_neg         1976 non-null float64
dtypes: float64(7), int64(1), object(9)
memory usage: 277.9+ KB


In [21]:
df.describe().loc[['min', 'max'], ['meta_score', 'user_score']]

,meta_score,user_score
min,0.0,0.0
max,97.0,9.5


In [22]:
df['n_user_score'] = df['user_score'] * 10

In [23]:
df['release_date'] = df['release_date'].apply(lambda x: pd.to_datetime(datetime.strptime(x, '%b %d, %Y')))

In [24]:
df.to_csv('xboxone_games_cleaned.csv', index=False, encoding = 'utf-8')

#### Switch

In [25]:
df = pd.read_csv('switch_games.csv', lineterminator='\n')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478 entries, 0 to 477
Data columns (total 17 columns):
title            478 non-null object
platform         478 non-null object
developer        477 non-null object
genre            478 non-null object
rating           440 non-null object
release_date     478 non-null object
summary          387 non-null object
meta_score       478 non-null int64
meta_overview    478 non-null object
meta_pos         478 non-null int64
meta_mixed       478 non-null int64
meta_neg         478 non-null int64
user_score       478 non-null object
user_overview    478 non-null object
user_pos         478 non-null object
user_mixed       478 non-null int64
user_neg         478 non-null object
dtypes: int64(5), object(12)
memory usage: 67.2+ KB


In [27]:
obj_to_numeric(df, ['meta_pos', 'meta_mixed', 'meta_neg', 'user_score', 'user_pos', 'user_mixed', 'user_neg'])

In [28]:
df.describe().loc[['min', 'max'], ['meta_score', 'user_score']]

,meta_score,user_score
min,0.0,0.0
max,97.0,9.3


In [29]:
df['n_user_score'] = df['user_score'] * 10

In [30]:
df['release_date'] = df['release_date'].apply(lambda x: pd.to_datetime(datetime.strptime(x, '%b %d, %Y')))

In [31]:
df.to_csv('switch_games_cleaned.csv', index=False, encoding = 'utf-8')

#### Merge

Let's merge the 3 .csv files into a single one called games.csv

In [36]:
consoles = ['ps4', 'xboxone', 'switch']

In [37]:
tables = [pd.read_csv(f'{c}_games_cleaned.csv', lineterminator='\n') for c in consoles]

In [38]:
for t in tables: print(t.shape)

(2704, 18)
(1976, 18)
(478, 18)


In [42]:
df = pd.concat(tables)

In [43]:
df.shape

(5158, 18)

In [46]:
df.to_csv('games.csv', index=False, encoding = 'utf-8')

## Reviews

#### Meta reviews

Load critics reviews of each platform

In [ ]:
meta_reviews = [pd.read_csv(f'{c}_meta_reviews.csv', lineterminator='\n') for c in consoles]

In [7]:
for t in meta_reviews: print(t.shape)

(33754, 6)
(13678, 6)
(4375, 6)


In [9]:
meta_reviews[0].head()

,score,text,critic,date,title,platform
0,74.0,"It makes a terrible first impression, but #kil...",Gaming Nexus,"Nov 18, 2014",#KILLALLZOMBIES,PlayStation 4
1,65.0,#killallzombies brings some fresh and inventiv...,Playstation Universe,"Nov 17, 2014",#KILLALLZOMBIES,PlayStation 4
2,60.0,A fun main mode of play that can get quite add...,Gamers' Temple,"Feb 16, 2015",#KILLALLZOMBIES,PlayStation 4
3,60.0,"Simply put, #killallzombies feels like half of...",DarkStation,"Dec 4, 2014",#KILLALLZOMBIES,PlayStation 4
4,60.0,"Combined with the price and lack of cross-buy,...",Worth Playing,"Nov 30, 2014",#KILLALLZOMBIES,PlayStation 4


Same analysis as we did with games info to find missing values and incorrect data formats

In [10]:
for t in meta_reviews: print(t.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33754 entries, 0 to 33753
Data columns (total 6 columns):
score       33753 non-null float64
text        33754 non-null object
critic      33754 non-null object
date        33754 non-null object
title       33754 non-null object
platform    33754 non-null object
dtypes: float64(1), object(5)
memory usage: 1.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13678 entries, 0 to 13677
Data columns (total 6 columns):
score       13678 non-null int64
text        13678 non-null object
critic      13678 non-null object
date        13678 non-null object
title       13678 non-null object
platform    13678 non-null object
dtypes: int64(1), object(5)
memory usage: 641.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4375 entries, 0 to 4374
Data columns (total 6 columns):
score       4375 non-null int64
text        4375 non-null object
critic      4375 non-null object
date        4375 non-null object
title       4375 non-null

In the first dataframe there's a null score

In [14]:
meta_reviews[0].loc[meta_reviews[0]['score'].isnull()]

,score,text,critic,date,title,platform
26547,NaN,I swear this is a real thing that is on the Pl...,The Jimquisition,"Jan 11, 2018",Sword of Fortress the Onomuzim,PlayStation 4


Let's delete its row

In [17]:
meta_reviews[0].drop(26547, inplace=True)

In [20]:
df = pd.concat(meta_reviews)

In [22]:
df['date'] = df['date'].apply(lambda x: pd.to_datetime(datetime.strptime(x, '%b %d, %Y')))

In [23]:
df.to_csv('meta_reviews.csv', index=False, encoding = 'utf-8')

#### User reviews

In [ ]:
user_reviews = [pd.read_csv(f'{c}_user_reviews.csv', lineterminator='\n') for c in consoles]

In [24]:
for t in user_reviews: print(t.shape)

(57237, 6)
(18840, 6)
(4802, 6)


In [25]:
user_reviews[0].head()

,score,text,critic,date,title,platform
0,9,For all (twin-stick) shooter lovers must have!...,413x,"Nov 3, 2014",#KILLALLZOMBIES,PlayStation 4
1,8,Simple twin stick shooter that throws enough r...,FriedConsole,"Apr 24, 2017",#KILLALLZOMBIES,PlayStation 4
2,10,Recién acabo de completar el volumen 1 y he de...,Evangelion37,"Dec 26, 2017",.hack//G.U. Last Recode,PlayStation 4
3,9,The best way to play the ps2 classic in this d...,Belial2Spooku,"Nov 3, 2017",.hack//G.U. Last Recode,PlayStation 4
4,9,.Hack//GU Last Recode is a great remaster and ...,Ffrenchie,"Nov 5, 2017",.hack//G.U. Last Recode,PlayStation 4


In [27]:
for t in user_reviews: print(t.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57237 entries, 0 to 57236
Data columns (total 6 columns):
score       57237 non-null int64
text        57236 non-null object
critic      57237 non-null object
date        57237 non-null object
title       57237 non-null object
platform    57237 non-null object
dtypes: int64(1), object(5)
memory usage: 3.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18840 entries, 0 to 18839
Data columns (total 6 columns):
score       18840 non-null int64
text        18840 non-null object
critic      18840 non-null object
date        18840 non-null object
title       18840 non-null object
platform    18840 non-null object
dtypes: int64(1), object(5)
memory usage: 1.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4802 entries, 0 to 4801
Data columns (total 6 columns):
score       4802 non-null int64
text        4802 non-null object
critic      4802 non-null object
date        4802 non-null object
title       4802 non-null objec

In [28]:
user_reviews[0].loc[user_reviews[0]['text'].isnull()]

,score,text,critic,date,title,platform
15918,9,NaN,Trefinn,"Nov 2, 2015",F1 2015,PlayStation 4


In [29]:
user_reviews[0].drop(15918, inplace=True)

In [30]:
df = pd.concat(user_reviews)

In [31]:
df['date'] = df['date'].apply(lambda x: pd.to_datetime(datetime.strptime(x, '%b %d, %Y')))

In [32]:
df.to_csv('user_reviews.csv', index=False, encoding = 'utf-8')

#### Merge

In [4]:
meta_df = pd.read_csv('meta_reviews.csv', lineterminator='\n')
user_df = pd.read_csv('user_reviews.csv', lineterminator='\n')

In [5]:
(meta_df.shape, user_df.shape)

((51806, 6), (80878, 6))

In [6]:
meta_df.describe().loc[['min', 'max'], ['score']]

,score
min,0.0
max,100.0


In [7]:
user_df.describe().loc[['min', 'max'], ['score']]

,score
min,0.0
max,10.0


As in games info, critics and user scores are in different range. 

In [8]:
user_df['score'] = user_df['score'] * 10

In [9]:
df = pd.concat([meta_df, user_df]).reset_index(drop=True)

Let's read some reviews

In [10]:
def print_examples(df, qty=1):
    for i in range(qty):
        print(df.iloc[i]['text'])
        print('\n')

In [11]:
print_examples(df.loc[df['score'] > 85], 5)

.hack//G.U. Last Recode had a lot of love poured into its development and planning because it looks amazing.


101 Ways To Die is great fun; each level is superbly crafted and the whole experience is enjoyable. Loading times are almost non-existent and the install size is only notable by the fact that it’s a mere 289mb. How they packed it all in to that, I’ll never understand.


2064: Read Only Memories presents an interesting, if not predictable, narrative that is propelled by its strong cast of characters and excellent world-building. It’s refreshing to see a developer put so much care into creating a cast as diverse and interesting as this.


It’s so purely fun. It’s a thrilling, charming, and humorous game that celebrates many positive things at once with an almost childlike innocence, all while containing a deep mystery that has you constantly thirsting for more information. After a difficult past year, it’s the type of thing that is sorely needed right now as we tackle this one w

In [12]:
print_examples(df.loc[df['score'] < 40], 5)

Sadly, 100ft Robot Golf is not the wonderful experience we expected it to be. A fantastic concept with poor execution.


2Dark will not make you feel annoyed, it will make you feel sad, and that's definitely worse. [May 2017, p.96]


2Dark tries to do too many things at once. Its stealth mechanics, while occasionally satisfying, are frustrating and pedestrian. Similarly, its attempts at horror, while presented well, are undermined by bad writing and repetitive gameplay. A lack of clear signposting and a terrible UI do nothing to help this maddening experience.


Visually and thematically, 2Dark succeeds in paying homage to its forebears. Everything else, however, is desperately lacking. Alone in the Dark had twenty years in which to rise and fall but Raynal’s latest stab at survival horror barely gets twenty minutes.


7 Days to Die has resulted in an infamous port from the PC game. It still features some good ideas like crafting or the combat system, but the technical aspect is full o

A lot of reviews contain the game name. This could be a problem by the time we train a sentiment analysis classifier. The algorithm could use that information to classify a review as positive or negative, something that won't be useful for new unseen data (of different games). Let's remove them (and also transform all texts to lower case)

In [13]:
df['text'] = df.apply(lambda x: x.text.lower().replace(f'{(x["title"]).lower()}', ''), 1)

Let's print the same previous examples

In [14]:
print_examples(df.loc[df['score'] < 40], 5)

sadly,  is not the wonderful experience we expected it to be. a fantastic concept with poor execution.


 will not make you feel annoyed, it will make you feel sad, and that's definitely worse. [may 2017, p.96]


 tries to do too many things at once. its stealth mechanics, while occasionally satisfying, are frustrating and pedestrian. similarly, its attempts at horror, while presented well, are undermined by bad writing and repetitive gameplay. a lack of clear signposting and a terrible ui do nothing to help this maddening experience.


visually and thematically,  succeeds in paying homage to its forebears. everything else, however, is desperately lacking. alone in the dark had twenty years in which to rise and fall but raynal’s latest stab at survival horror barely gets twenty minutes.


 has resulted in an infamous port from the pc game. it still features some good ideas like crafting or the combat system, but the technical aspect is full of bugs and graphics issues.




Printing reviews I noticed that some of them are not english. For example:

In [15]:
df.iloc[132654]['text']

'133小时完结散花，\r 画面、新手教学、地图指引不好，\r 音乐、过场演出、地图设计、剧情和设定很棒，\r 掌机模式降到300p左右分辨率依然跑图跑的很开心就tmd很奇葩，\r 剩下还有一堆东西要肝，全部打完估计一共200小时了，\r 今年能玩到p5和xb2真是太好了，\r 我永远喜欢吼姆拉.jpg'

In [16]:
df.iloc[132648]['text']

'magistral!. para mi el mejor juego del año, hace tiempo no me enganchaba tanto con un vídeo-juego, el sistema de combate es perfecto los mundos son inmensos y con una puesta en escena impresionante. el único punto en contra que le veo es que algunas, no todas, misiones secundarias son un poco aburridas. pero la historia principal está a un muy alto nivel.\rdefinitivamente una obra maestra!'

The langdetect library is useful to find the language of a given text

In [ ]:
def detect_lang(row):
    try:
        lang = detect(row.text)
    except:
        lang = "error"
    return lang

In [ ]:
df['lang'] = df.apply(lambda x: detect_lang(x), 1)

In [18]:
df['lang'].value_counts()[:10]

en    127699
es      2201
pt       868
ru       596
fr       306
de       238
it       169
tr       113
pl        86
so        47
Name: lang, dtype: int64

We are only interested in english reviews

In [ ]:
df = df.loc[df['lang'] == 'en']

In [ ]:
df.to_csv('reviews.csv', index=False, encoding = 'utf-8')